In [8]:
import sqlite3
import geojson as gj


In [66]:
db_filename = 'data/arbres.db'

db = sqlite3.connect(db_filename)

# -- create the table --
cursor = db.cursor()

In [73]:
cursor.execute("""
    SELECT arbres.genre_bota, arbres.espece, arbres.variete, nomsfr.nomfr,
                arbres.anneedeplantation, arbres.stadededeveloppement,
                arbres.sous_categorie, arbres.sous_categorie_desc,
                arbres.latitude, arbres.longitude,
                arbres.code_parent_desc, arbres.code_parent
    FROM arbres
    INNER JOIN nomsfr ON arbres.genre_bota IS nomsfr.genre_bota
                                    AND  arbres.espece IS nomsfr.espece
                                    AND  arbres.variete IS nomsfr.variete
             """)

field_name = [ desc[0].upper() for desc in cursor.description ]
print( field_name )

['GENRE_BOTA', 'ESPECE', 'VARIETE', 'NOMFR', 'ANNEEDEPLANTATION', 'STADEDEDEVELOPPEMENT', 'SOUS_CATEGORIE', 'SOUS_CATEGORIE_DESC', 'LATITUDE', 'LONGITUDE', 'CODE_PARENT_DESC', 'CODE_PARENT']


In [74]:
# remplace None by an empty string :
emptystr = lambda x: x if x else ''

In [75]:
N_iter_max = 50000

collection_list = []
for index in range(N_iter_max):
    row = cursor.fetchone()
    if not row:
        break
    
    props = { k:emptystr(v) for k, v in zip( field_name, row ) }
    coords = [ props[k] for k in ['LONGITUDE', 'LATITUDE'] ]
    
    point = gj.Point( coords )
    feature = gj.Feature( geometry=point, properties=props )
    
    collection_list.append( feature )

    
feat_collection = gj.FeatureCollection( collection_list )
print( len(collection_list) )

# - save -
geonjson_text = gj.dumps( feat_collection )
print(len(geonjson_text))

filename = 'data/arbres_transformed.geojson'
with open(filename, '+w') as file:
    file.write( geonjson_text )

30913
15171773
